In [ ]:
# Running this notebook on Google Colab? Then run this cell! Otherwise, please use the cell above!

!pip install selenium
!pip install helium
!apt-get update 
!apt install chromium-chromedriver
!pip install requests


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from datetime import date

import pandas as pd
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from helium import *
import requests


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [ ]:
url = 'https://fairfaxcryobank.com/search/searchresults.aspx'
driver.get(url)
element = driver.find_element(By.CLASS_NAME, "submit")
element.click()


In [ ]:
results_html = BeautifulSoup(driver.page_source, 'html.parser')
h = results_html.find_all('div', attrs={'class': 'total'})[0]
pages = h.get_text()[-2:]
pages = int(pages)
pages

36

In [ ]:
def next_page():
  #driver.find_element(By.XPATH,"//*[@id='content']/div/div[3]/div/div[1]/div[4]/div[2]/a").click()
  #results_html = BeautifulSoup(driver.page_source, 'html.parser')
  try:
    WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='content']/div/div[3]/div/div[1]/div[4]/div[2]/a"))).click()
  except:
    pass

In [ ]:
#initiate dataframe 
df_list=[]


In [ ]:
def page_data():
  results_html = BeautifulSoup(driver.page_source, 'html.parser')
  #results_html.find_all('div', attrs={'class': 'bl-03'})[0]
  bl01 = [x.get_text() for x in results_html.find_all('div', attrs={'class': 'bl-01'})] #profile number and description
  bl02 = [x.get_text() for x in results_html.find_all('div', attrs={'class': 'bl-02'})] #blood and clickable iui and ivf etc
  bl03 = [x.get_text() for x in results_html.find_all('div', attrs={'class': 'bl-03'})] #ancestry ethnic background, eyes hair height and a lot
  bl04 = [x.get_text() for x in results_html.find_all('div', attrs={'class': 'bl-04'})] #tested for diseases etc
#bl05 = [x.get_text() for x in results_html.find_all('div', attrs={'class': 'bl-05'})] doesn't exist
  bl06 = [x.get_text() for x in results_html.find_all('div', attrs={'class': 'bl-06'})] #brand

  #height and ancestry
  arr = [a.split("\n") for a in bl03]
  blocks = len(arr)
  #print(blocks)
  for i in range(1):
    data = {f'{arr[i][4]} Mat':arr[i][10],
          f'{arr[i][4]} Pat':[arr[i][11]],
          f'{arr[i][19]}':arr[i][28], 
          f'{arr[i][20]}':arr[i][29], 
          f'{arr[i][21]}':arr[i][30],
          f'{arr[i][22]}':arr[i][31],
          f'{arr[i][40]}':arr[i][46], 
          f'{arr[i][41]}':arr[i][47]
        }
  df = pd.DataFrame(data)
  for j in range(1,blocks):
    df.loc[j] = [arr[j][10],
              arr[j][11],
              arr[j][28],
              arr[j][29],
              arr[j][30],
              arr[j][31],
              arr[j][46],
              arr[j][47]]
              
             
  #bl01 profile,  etc
  arr = [a.split("\n") for a in bl01]

  for i in range(1):
    data = {'Profile':[arr[i][1][0:4]]
        }
        
  df1 = pd.DataFrame(data)
  for j in range(1,blocks):
    df1.loc[j] = arr[j][1][0:4]

  df_final = pd.merge(df,df1,left_index=True, right_index=True)


#bl02 blood type
  arr = [a.split("\n") for a in bl02]
  for i in range(1):
    data = {'Blood Type':[arr[i][18]]
          }     
  df = pd.DataFrame(data)
  for j in range(blocks):
   df.loc[j] = arr[j][18]


  df_final = pd.merge(df_final,df,left_index=True, right_index=True)
  list = [bl01, arr, bl02, bl03, df, df1, results_html]
  del list
  return df_final

In [ ]:
#h = results_html.find_all('div', attrs={'class': 'bl-03'})[0]
#h.get_text()



In [ ]:


for i in range(1):
  df_final = page_data()
  df_list = df_final
  del [df_final]

df_list = df_list.drop(range(0,10))
for j in range(1,pages+1):
  next_page()
  df_final = page_data()
  df_list = pd.concat([df_list,df_final])
  del df_final
 
now = datetime.now()
df_list['Date']= now
df_list




,Ancestry Mat,Ancestry Pat,Height,Eyes,Hair,CMV,Occupation,Degree,Profile,Blood Type,Date
0,Asian,Nepalese/Nepalese,"5'8""(172 cm)",Brown,BlackStraight,Positive,Postdoctoral fellow,PhD/Analytical Chemistry,6281,A+,2022-02-16 18:37:20.260942
1,Multi,Mexican/English-Mexican,"6'0""(182 cm)",Brown,BlackStraight,Positive,Engineer,MS/Engineering Management,6087,O+,2022-02-16 18:37:20.260942
2,Asian,Vietnamese/Vietnamese,"5'9""(175 cm)",Brown,BlackStraight,Positive,Postdoctoral researcher,PhD/Material Sci & Engineering,5866,O+,2022-02-16 18:37:20.260942
3,Asian,Chinese/Chinese,"5'10""(177 cm)",Brown,BlackStraight,Positive,Data Scientist/Programmer Anal,MS/Math and Computational Fin,5852,B+,2022-02-16 18:37:20.260942
4,Latino,Mexican/Mexican,"6'0""(182 cm)",Brown,BlackWavy,Positive,Student,BA/Psyc & Spanish,5834,O+,2022-02-16 18:37:20.260942
...,...,...,...,...,...,...,...,...,...,...,...
1,Asian,Chinese/Chinese,"5'9""(175 cm)",Brown,BlackStraight,Positive,Consulting Firm Partner,PhD/Computer Science,1886,O+,2022-02-16 18:37:20.260942
2,Asian,Chinese/Chinese,"5'9""(175 cm)",Brown,BlackStraight,Negative,Student,PhD/Oncology,1862,AB+,2022-02-16 18:37:20.260942
3,Multi,Thai/Slovenian,"6'3""(190 cm)",Brown,BlackWavy,Positive,Server/ Bartender / Musician,BA/Tourism,9364,O+,2022-02-16 18:37:20.260942
4,Caucasian,English-French/German-Scottish,"6'0""(182 cm)",Brown,BrownStraight,Negative,Litigation Support/Law Student,JD/Law,6238,AB+,2022-02-16 18:37:20.260942


In [ ]:
df_list.head(20)

,Ancestry Mat,Ancestry Pat,Height,Eyes,Hair,CMV,Occupation,Degree,Profile,Blood Type,Date
0,Asian,Nepalese/Nepalese,"5'8""(172 cm)",Brown,BlackStraight,Positive,Postdoctoral fellow,PhD/Analytical Chemistry,6281 Profile,A+,2022-02-16 18:29:52.363985
1,Multi,Mexican/English-Mexican,"6'0""(182 cm)",Brown,BlackStraight,Positive,Engineer,MS/Engineering Management,6087,O+,2022-02-16 18:29:52.363985
2,Asian,Vietnamese/Vietnamese,"5'9""(175 cm)",Brown,BlackStraight,Positive,Postdoctoral researcher,PhD/Material Sci & Engineering,5866,O+,2022-02-16 18:29:52.363985
3,Asian,Chinese/Chinese,"5'10""(177 cm)",Brown,BlackStraight,Positive,Data Scientist/Programmer Anal,MS/Math and Computational Fin,5852,B+,2022-02-16 18:29:52.363985
4,Latino,Mexican/Mexican,"6'0""(182 cm)",Brown,BlackWavy,Positive,Student,BA/Psyc & Spanish,5834,O+,2022-02-16 18:29:52.363985
5,Asian,Chinese/Chinese,"6'0""(182 cm)",Brown,BlackStraight,Positive,Investment Strategist,MBA,6331,AB+,2022-02-16 18:29:52.363985
6,Caucasian,German-Norwegian/German-Norwegian,"5'10""(177 cm)",Blue,BrownCurly,Positive,Facility Manager,Bachelor of Fine Arts,6426,O+,2022-02-16 18:29:52.363985
7,Multi,Filipino/English-Irish,"5'7""(170 cm)",Brown,BrownStraight,Positive,Student,Master of Public Affairs,5619,O-,2022-02-16 18:29:52.363985
8,Caucasian,Dutch/Polish (Jewish),"5'9""(175 cm)",Hazel,BrownStraight,Negative,Research Assistant,BA/Theatre Arts,6414,O+,2022-02-16 18:29:52.363985
9,Asian,Chinese/Chinese,"5'7""(170 cm)",Brown,BlackStraight,Positive,Personal Trainer,BS/Computer Science,5595,B+,2022-02-16 18:29:52.363985


In [ ]:
today = datetime.now()
date1 = today.strftime("%b-%d")
from google.colab import files
df_list.to_csv('%s.csv'%date1) 
files.download('%s.csv'%date1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>